# Using the ``fancytypes.Updatable`` class #

## A NOTE BEFORE STARTING ##

Since the ``fancytypes`` git repository tracks this notebook under its original
basename ``using_Updatable_class.ipynb``, we recommend that you copy the
original notebook and rename it to any other basename that is not one of the
original basenames that appear in the ``<root>/examples`` directory before
executing any of the notebook cells below, where ``<root>`` is the root of the
``fancytypes`` repository. This way you can explore the notebook by executing
and modifying cells without changing the original notebook, which is being
tracked by git.

## Table of contents ##

- [Import necessary modules](#Import-necessary-modules)
- [Introduction](#Introduction)
- [Defining the ``SliceShuffler`` class](#Defining-the-SliceShuffler-class)
- [Using the ``SliceShuffler`` class](#Using-the-SliceShuffler-class)

## Import necessary modules ##

In [1]:
# For performing deep copies.
import copy



# For general array handling and constructing random number generators.
import numpy as np

# For validating and converting objects.
import czekitout.check
import czekitout.convert



# The library that is the subject of this demonstration.
import fancytypes

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Introduction ##

In this notebook, we use the ``fancytypes.Updatable`` class to define a class of
"slice shufflers", which we define as objects that can shuffle/re-order the
elements in a slice of a given array. This is a somewhat contrived example use
of the ``fancytypes.Updatable`` class, however it is simple and complete. You 
can find the documentation for the ``fancytypes.Updatable`` class
[here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.Updatable.html).
It is recommended that you consult the documentation of this class as you 
explore the notebook.

This notebook also demonstrates how one can use the helper function 
``fancytypes.return_validation_and_conversion_funcs``. We use this function to
define our class of slice shufflers. You can find the documentation for this
helper function
[here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.return_validation_and_conversion_funcs.html).

**Users should make sure to navigate the documentation for the version of
fancytypes that they are currently using.**

In order to execute the cells in this notebook as intended, a set of Python
libraries need to be installed in the Python environment within which the cells
of the notebook are to be executed. For this particular notebook, users need to
install:

    fancytypes
    jupyter

Users can install these libraries either via `pip`:

    pip install fancytypes[examples]

or `conda`:

    conda install -y fancytypes jupyter -c conda-forge

## Defining the ``SliceShuffler`` class ##

We define the ``SliceShuffler`` class as a subclass of the
``fancytypes.Updatable`` class.

In [2]:
# Define the validation and conversion functions.
def _check_and_convert_slice_obj(params):
    obj_name = "slice_obj"
    kwargs = {"obj": params[obj_name],
              "obj_name": obj_name,
              "accepted_types": (slice,)}
    czekitout.check.if_instance_of_any_accepted_types(**kwargs)
    slice_obj = copy.deepcopy(params[obj_name])

    return slice_obj

def _check_and_convert_seed(params):
    obj_name = "seed"
    kwargs = {"obj": params[obj_name], "obj_name": obj_name}
    seed = czekitout.convert.to_nonnegative_int(**kwargs)

    return seed



# Define the ``SliceShuffler`` class.
class SliceShuffler(fancytypes.Updatable):
    ctor_param_names = ("slice_obj", "seed")
    kwargs = {"namespace_as_dict": globals(),
              "ctor_param_names": ctor_param_names}

    _validation_and_conversion_funcs_ = \
        fancytypes.return_validation_and_conversion_funcs(**kwargs)

    del ctor_param_names, kwargs

    def __init__(self, slice_obj, seed, skip_validation_and_conversion=False):
        ctor_params = {key: val
                       for key, val in locals().items()
                       if (key not in ("self", "__class__"))}

        # Set ``skip_cls_tests`` to ``True`` only if you are sure that the
        # class that you have defined was done so properly, i.e. without errors.
        skip_cls_tests = True

        kwargs = ctor_params
        kwargs["skip_cls_tests"] = skip_cls_tests
        fancytypes.Updatable.__init__(self, **kwargs)

        self.execute_post_core_attrs_update_actions()

        return None

    def execute_post_core_attrs_update_actions(self):
        seed = self.core_attrs["seed"]
        self._random_generator = np.random.default_rng(seed)

        return None

    @classmethod
    def get_validation_and_conversion_funcs(cls):
        validation_and_conversion_funcs = \
            cls._validation_and_conversion_funcs_.copy()

        return validation_and_conversion_funcs

    def shuffle(self, array):
        try:
            array = np.array(array)
        except:
            err_msg = ("The object ``array`` must be array-like.")
            raise TypeError(err_msg)
            
        slice_obj = self.core_attrs["slice_obj"]
        array_slice = array[slice_obj]
        self._random_generator.shuffle(array_slice)
        array[slice_obj] = array_slice

        return array

    # Overriding the method ``fancytypes.Updatable.update`` such that additional
    # steps are performed after updating any core attributes.
    def update(self, 
               new_core_attr_subset_candidate, 
               skip_validation_and_conversion=False):
        kwargs = {key: val
                  for key, val in locals().items()
                  if (key not in ("self", "__class__"))}
        fancytypes.Updatable.update(self, **kwargs)
        self.execute_post_core_attrs_update_actions()

        return None

## Using the ``SliceShuffler`` class ##

First let's construct a valid instance of the ``SliceShuffler`` class with 
validation and conversion of the parameters to be mapped to the 
"core attributes" enabled.

In [3]:
kwargs = {"slice_obj": slice(None, 6, 1), 
          "seed": 5.0, 
          "skip_validation_and_conversion": False}
slice_shuffler = SliceShuffler(**kwargs)

There are 3 ways of accessing the core attributes via the public API:

In [4]:
# Returns a deep copy.
core_attrs = slice_shuffler.core_attrs
print(core_attrs)

# Returns a deep copy.
core_attrs = slice_shuffler.get_core_attrs(deep_copy=True)
print(core_attrs)

# Returns a reference, i.e. no copy is made. 
core_attrs = slice_shuffler.get_core_attrs(deep_copy=False)
print(core_attrs)

{'slice_obj': slice(None, 6, 1), 'seed': 5}
{'slice_obj': slice(None, 6, 1), 'seed': 5}
{'slice_obj': slice(None, 6, 1), 'seed': 5}


There is 1 way of accessing the validation and conversion functions via the 
public API:

In [5]:
# Returns a deep copy.
validation_and_conversion_funcs = \
    slice_shuffler.validation_and_conversion_funcs

print(validation_and_conversion_funcs)

{'slice_obj': <function _check_and_convert_slice_obj at 0x7f5ca1b56840>, 'seed': <function _check_and_convert_seed at 0x7f5ca1b568e0>}


Next let's construct a valid instance of the SliceShuffler class with validation
and conversion of the parameters to be mapped to the core attributes disabled. 
This option is desired primarily when the user wants to avoid potentially 
expensive copies and/or conversions of the parameters to be mapped to the core 
attributes. However, users must ensure that the construction parameters are 
valid and require no conversions.

In [6]:
kwargs = {"slice_obj": slice(None, 6, 1), 
          "seed": 5,  # Manually converted parameter to a `int` as required.
          "skip_validation_and_conversion": True}
slice_shuffler = SliceShuffler(**kwargs)

core_attrs = slice_shuffler.core_attrs
print(core_attrs)

{'slice_obj': slice(None, 6, 1), 'seed': 5}


Let's shuffle an array using the class.

In [7]:
array = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

array([1, 4, 2, 3, 5, 0, 6, 7, 8, 9])

Let's update the slice shuffler with validations and conversions enabled. Again,
users can optionally disable validations and conversions to avoid potentially
expensive copies and/or conversions.

In [8]:
new_core_attr_subset_candidate = {"slice_obj": slice(3, None, 1)}
kwargs = {"new_core_attr_subset_candidate": new_core_attr_subset_candidate,
          "skip_validation_and_conversion": False}
slice_shuffler.update(**kwargs)

print(slice_shuffler.core_attrs)

{'slice_obj': slice(3, None, 1), 'seed': 5}


Let's shuffle the original array again.

In [9]:
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

array([0, 1, 2, 4, 7, 5, 6, 9, 8, 3])

Let's update the slice shuffler again.

In [10]:
new_core_attr_subset_candidate = {"slice_obj": slice(3, 7, 1), "seed": 2}
kwargs = {"new_core_attr_subset_candidate": new_core_attr_subset_candidate,
          "skip_validation_and_conversion": True}
slice_shuffler.update(**kwargs)

print(slice_shuffler.core_attrs)

{'slice_obj': slice(3, 7, 1), 'seed': 2}


Let's shuffle the original array again.

In [11]:
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

array([0, 1, 2, 6, 5, 3, 4, 7, 8, 9])

Let's try constructing instances of the ``SliceShuffler`` class with invalid
construction parameters. The following two codes blocks return errors as 
expected:

In [12]:
# Note that by default ``skip_validation_and_conversion == False``.
kwargs = {"slice_obj": 3, "seed": 5.0}
slice_shuffler = SliceShuffler(**kwargs)

TypeError: The object ``slice_obj`` must be an instance of the class `slice`.

In [13]:
kwargs = {"slice_obj": slice(None, 6, 1), "seed": "foo"}
slice_shuffler = SliceShuffler(slice_obj=slice(None, 6, 1), seed="foo")

TypeError: The object ``seed`` must be an integer.

Note that being a direct subclass of the ``fancytypes.Updatable`` class, the
``SliceShuffler`` class supports updatable core attributes, and validation upon
construction and updates, but it does not support pre-serialization, or
de-pre-serializable.

The ``fancytypes.Checkable`` class only supports validation upon constructing
instances.

The ``fancytypes.PreSerializable`` class supports pre-serialization,
de-serialization, and enforces validation upon constructing instances, but it
does not support updatable core attributes.

The ``fancytypes.PreSerializableAndUpdatable`` class supports pre-serialization,
de-serialization, updatable core attributes, and enforces validation upon
constructing or updating instances.